# Sudoku Solver

Write a program to solve a Sudoku puzzle by filling the empty cells.

A sudoku solution must satisfy all of the following rules:

- Each of the digits 1-9 must occur exactly once in each row.
- Each of the digits 1-9 must occur exactly once in each column.
- Each of the digits 1-9 must occur exactly once in each of the 9 3x3 sub-boxes of the grid.

The '.' character indicates empty cells.

In [2]:
class Solution:
    def solveSudoku(self, board: list[list[str]]) -> None:
        self.solve(board)
    
    def solve(self, board: list[list[str]]) -> bool:
        for i in range(9):
            for j in range(9):
                if board[i][j] == '.':
                    for num in '123456789':
                        if self.isValid(board, i, j, num):
                            board[i][j] = num
                            if self.solve(board):
                                return True
                            board[i][j] = '.'
                    return False
        return True
    
    def isValid(self, board: list[list[str]], row: int, col: int, num: str) -> bool:
        for i in range(9):
            if board[row][i] == num:
                return False
            
        for i in range(9):
            if board[i][col] == num:
                return False
        
        box_row, box_col = 3 * (row // 3), 3 * (col // 3) 
        for i in range(3):
            for j in range(3):
                if board[box_row + i][box_col + j] == num:
                    return False
        return True 

In [4]:
class Solution:
    def solveSudoku(self, board: list[list[str]]) -> None:
        def init_state():
            rows = [0] * 9
            cols = [0] * 9
            boxes = [0] * 9
            empty = []
            for i in range(9):
                for j in range(9):
                    if board[i][j] != ".":
                        num = int(board[i][j]) - 1
                        rows[i] |= 1 << num
                        cols[j] |= 1 << num
                        boxes[(i // 3) * 3 + j // 3] |= 1 << num
                    else:
                        empty.append((i, j))
            return rows, cols, boxes, empty

        def get_possible(i, j):
            box_idx = (i // 3) * 3 + j // 3
            return ~(rows[i] | cols[j] | boxes[box_idx]) & 0x1FF

        def solve():
            if not empty:
                return True

            # 選擇可能值最少的格子
            i, j = min(empty, key=lambda x: bin(get_possible(x[0], x[1])).count("1"))
            possible = get_possible(i, j)

            if not possible:
                return False

            box_idx = (i // 3) * 3 + j // 3
            empty.remove((i, j))

            while possible:
                num = possible & -possible
                possible &= possible - 1
                if rows[i] & num or cols[j] & num or boxes[box_idx] & num:
                    continue

                rows[i] |= num
                cols[j] |= num
                boxes[box_idx] |= num
                board[i][j] = str(bin(num).count("0"))

                if solve():
                    return True

                rows[i] &= ~num
                cols[j] &= ~num
                boxes[box_idx] &= ~num

            empty.append((i, j))
            board[i][j] = "."
            return False

        rows, cols, boxes, empty = init_state()
        solve()